In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
!aws configure set aws_access_key_id "$AWS_ACCESS_KEY_ID"
!aws configure set aws_secret_access_key "$AWS_SECRET_ACCESS_KEY" 
!aws configure set region "us-east-1" 
!aws configure set output "text"



In [3]:
%%bash

bucketname="$(echo $USER"-sql-workshop"|cut -d "-" -f2-)"
echo $bucketname

cp /etc/skel/.env ~/
echo "BUCKETNAME="$bucketname >> ~/.env

aws s3api create-bucket --bucket $bucketname --region us-east-1
aws s3api list-buckets | grep $bucketname 

testing4-sql-workshop
/testing4-sql-workshop
BUCKETS	2021-09-15T15:19:44.000Z	testing4-sql-workshop


In [ ]:
#Use this code block by removing the "#" in front of the following commands 
#After the workshop to set up credentials for your AWS Account To use this Notebook
#This has already been done on the workshop server

#%env AWS_ACCESS_KEY_ID=<your_AWS_Access ID>
#%env AWS_SECRET_ACCESS_KEY=<your AWS Secret>
#%env BUCKETNAME=<your staging bucket name>


#!aws configure set aws_access_key_id "$AWS_ACCESS_KEY_ID"
#!aws configure set aws_secret_access_key "$AWS_SECRET_ACCESS_KEY" 
#!aws configure set region "us-east-1" 
#!aws configure set output "text"


#!sudo pip install pyathena 
#!sudo pip install ipython-sql
#!pip install sqlalchemy

